In [1]:
from HelperClass import HTMLTokenizer, InvertedIndex
from helper import cleanHtml

In [2]:
import os, json

jsonFolder = 'Json/'
jsonFiles = [pos_json for pos_json in os.listdir(jsonFolder) if pos_json.endswith('.json')]
print(jsonFiles)

['0f274aaa945c05641a9677b951c32026bb201ec9aeb6e691fedd1235b3a5d6af.json', '192286a9954a2917a50ad6d5bb1efa61e2de5e94c7e9763d0d3c6e985677c6a5.json', '2b8802db76f30d1ea6cfc23048d6513e8a23dbf307976fff915062100762d99f.json', '35f503672017253c81058a83bc9ad0adfe3aa6f3933a4f232a93b6a5cdfd28cf.json', '7a23de605f63cf4d997c135aec35ffca3ddef439c383a7cab9b149e829d4e3f6.json', '8ef6d99d9f9264fc84514cdd2e680d35843785310331e1db4bbd06dd2b8eda9b.json', '906c24a2203dd5d6cce210c733c48b336ef58293212218808cf8fb88edcecc3b.json', '9a59f63e6facdc3e5fe5aa105c603b545d4145769a107b4dc388312a85cf76d5.json', 'c6e615a6a72d7518bd77857d8651f2c04b35207d9039ebf3cc1509ea76202013.json', 'e3554fad67505553a9742d31c4b526701f9f4a8a0a0eedfd9dafee6af43d59fc.json']


In [3]:
def getHtmlContent(jsonFile:str):
    jFile = open('Json/'+jsonFile)
    data = json.load(jFile)
    jFile.close()

    #Get encoding and html content
    e = data['encoding']
    jFile = open('Json/'+jsonFile, encoding=e)
    data = json.load(jFile)
    jFile.close()
    return data['content']

In [4]:
def geturl(jsonFile:str):
    jFile = open('Json/'+jsonFile)
    data = json.load(jFile)
    jFile.close()
    return data['url']

In [5]:
urlList = []
for file in jsonFiles:
    urlList.append(geturl(file))

In [17]:
invIndex = InvertedIndex()
docIdCount = 1
for file in jsonFiles:
    htmlContent = getHtmlContent(file)
    parser = HTMLTokenizer() #Create parser obj
    parser.feed(cleanHtml(htmlContent)) #pass in clean html to parse
    parser.getDoc().setDocId(docIdCount) #Set docId
    doc = parser.getDoc()
    parser.clear() #reset parser to parse new doc
    invIndex.addDoc(doc)
    docIdCount += 1

In [19]:
def getAllTokens(invIndex):
    tokens = set()
    for i in range(1, invIndex.getSize()+1):
        for tok in invIndex[i]._postings:
            tokens.add(tok)
    return tokens

len(getAllTokens(invIndex))

713

In [8]:
def getWeightStr(invIndex):
    wStr = '| Weights | '
    for i in range(1, invIndex.getSize()+1):
        wCount = 0
        for w in invIndex[i].getWeights():
            wStr += f'{w}: '
            pCount = 0
            for p in invIndex[i].getWeights()[w]:
                wStr += f'{p}'
                pCount += 1
                if pCount > 2:
                    wStr += '..'
                    break
                wStr += ','
            wStr = wStr.strip(',') + ' '
            wCount += 1
            if wCount > 1:
                break
        wStr += '|'
        
    return wStr

In [9]:
def getTokenStr(invIndex, token):
    tStr = token + ' Freq: '
    pos = 0
    fStr = '| '
    for i in range(1, invIndex.getSize()+1):
        pSize = len(invIndex[i][token]['pos'])
        pos += pSize
        fStr += f'Freq: {pSize} '
        pCount = 0
        if pSize > 0:
            fStr += 'Pos: '
        for p in invIndex[i][token]['pos']:
            pCount += 1
            fStr += f'{p},'
            if pCount > 2:
                fStr = fStr.strip(',')
                fStr += '...'
                break
        fStr.strip(',')
        fStr += '| '

    return tStr + f'{pos} ' + fStr

In [10]:
def getAllTok(invIndex):
    tStr = ''
    for tok in getAllTokens(invIndex):
        if not tStr:
            tStr += f'| Tokens  | {getTokenStr(invIndex, tok)}'
        else:
            tStr += f'|  | {getTokenStr(invIndex, tok)}'
        tStr += '\n'
    return tStr

In [11]:
def getUrlStr(urlList):
    uStr = '| DocId | Url |\n|-------|-----|\n'
    count = 1
    for url in urlList:
        uStr += f'| {count} | {url} |'
        count += 1
        uStr += '\n'
    uStr += '\n\n'
    return uStr

In [12]:

reportFile = open("Report/Report.md", "w")
reportFile.write(getUrlStr(urlList)+'\n')
reportFile.close()

In [13]:
reportFile = open("Report/Report.md", "a")
dStr = '| DocId'
for i in range(1, invIndex.getSize()+1):
    dStr += f' | {i}'
dStr += ' |'
reportFile.write(dStr+'\n')
dStr = '|'
for i in range(0, invIndex.getSize()+1):
    dStr += '-----'
    dStr += ' |'
reportFile.write(dStr+'\n')
reportFile.close()

In [14]:
reportFile = open("Report/Report.md", "a")
reportFile.write(getWeightStr(invIndex)+'\n')
reportFile.close()

In [15]:
reportFile = open("Report/Report.md", "a")
reportFile.write(getAllTok(invIndex)+'\n')
reportFile.close()